In [5]:
import pandas as pd
from google.cloud import bigquery
import db_dtypes
import time
import numpy as np
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [6]:
TIME_PERIOD = 5
PATH_CRED = 'project-test-newage-1dd885e89df4.json'

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(PATH_CRED, scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '1oOGZsWFLrvzILsv-IxI9Sa6hhRrZhqvBm3F19LvHpfQ'


query_arr = []
for i in range(1, TIME_PERIOD + 1):
    query_arr.append(f"""SELECT * FROM `bigquery-public-data.google_analytics_sample.ga_sessions_2016080{i}`""")

In [7]:
def fetch_queries(path, queries, TIME_PERIOD = TIME_PERIOD):
    client = bigquery.Client.from_service_account_json(json_credentials_path=path)
    result = []
    print('-------------------------')
    print('Start fetching queries:')
    start_time = time.time()
    for query in queries:
        job = client.query(query).result()
        result.append(job)
        print("Processing...")

    print('all jobs done!')
    print(f"Time for fetching {TIME_PERIOD} queries: {round(time.time() - start_time, 2)} seconds")
    print('-------------------------')
    return result

def make_df(jobs):
    print('-------------------------')
    print("Start concat main data frame:")
    start_time = time.time()
    df_arr = [j.to_dataframe() for j in jobs]
    df = pd.concat(df_arr, ignore_index=True)
    print(f"Time for making main df: {round(time.time() - start_time, 2)} seconds")
    print('-------------------------')
    return df
    
    
def unpack_df(df):
    start_time = time.time() 
    print('-------------------------')
    print('Unpacking main data frame:')
    columns = ['totals', 'trafficSource', 'device', 'geoNetwork', 'customDimensions']
    unp_arr = []
    for col in columns:
        unp_arr.append(df[col].apply(pd.Series))
        print("Processing...")
    struct_df = [df.visitNumber, df.visitId, df.visitStartTime, df.date, unp_arr[0],\
                 unp_arr[1], unp_arr[2], unp_arr[3],\
                 unp_arr[4], df.fullVisitorId, df.channelGrouping, df.socialEngagementType]
    new_df = pd.DataFrame(pd.concat(struct_df, axis = 1)).reset_index()
    print(f"Time for unpacking main df: {round(time.time() - start_time, 2)} seconds")
    print('-------------------------')
    return new_df


In [8]:
global_time = time.time()

print('-------------------------')
print('START OF SCRIPT EXECUTION:')


main_df = make_df(fetch_queries(path = PATH_CRED, queries = query_arr))

df = unpack_df(main_df) #without hits, userId, visitorId

agg_time = time.time()
print('-------------------------')
print('Creating result data frames:')

#df_1 - number of total visits by date
df_1 = pd.DataFrame(main_df.groupby(['date'])['totals'].count().reset_index()) #working with main_df
df_1.rename(columns={"totals": "total_visits"}, inplace=True)

#df_2 - mean hits, mean time on site, max hits, max time on site by city
df_2 = df.groupby('city').agg(
    mean_hits = ('hits', 'mean'), 
    mean_timeOnSite=('timeOnSite', 'mean'),
    max_hits = ('hits', 'max'), 
    max_timeOnSite=('timeOnSite', 'max'),
    ).reset_index()

#df_3 - total value of time on site grouped by operating system and browser
df_3 = df.groupby(['operatingSystem', 'browser'])['timeOnSite'].sum().reset_index()


print(f"Time for creating resulting data frames: {round(time.time() - agg_time, 2)} seconds")
print('-------------------------')

print('-------------------------')
print('Uploading data frames to Google Sheets:')
upload_time = time.time()

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(PATH_CRED, scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '1oOGZsWFLrvzILsv-IxI9Sa6hhRrZhqvBm3F19LvHpfQ'

d2g.upload(df_1, spreadsheet_key, 'df_1_result', credentials=credentials, row_names=False)
print('First data frame')

d2g.upload(df_2, spreadsheet_key, 'df_2_result', credentials=credentials, row_names=False)
print('Second data frame')

d2g.upload(df_3, spreadsheet_key, 'df_3_result', credentials=credentials, row_names=False)
print('Third data frame')

print(f"Time for uploading dfs: {round(time.time() - upload_time, 2)} seconds")
print('-------------------------')

print(f"TIME FOR EXECUTION WHOLE SCRIPT: {round(time.time() - global_time, 2)} seconds")
print('-------------DONE------------')

-------------------------
START OF SCRIPT EXECUTION:
-------------------------
Start fetching queries:
Processing...
Processing...
Processing...
Processing...
Processing...
all jobs done!
Time for fetching 5 queries: 5.45 seconds
-------------------------
-------------------------
Start concat main data frame:
Time for making main df: 29.86 seconds
-------------------------
-------------------------
Unpacking main data frame:
Processing...
Processing...
Processing...
Processing...
Processing...
Time for unpacking main df: 29.82 seconds
-------------------------
-------------------------
Creating result data frames:
Time for creating resulting data frames: 0.02 seconds
-------------------------
-------------------------
Uploading data frames to Google Sheets:
First data frame
Second data frame
Third data frame
Time for uploading dfs: 11.58 seconds
-------------------------
TIME FOR EXECUTION WHOLE SCRIPT: 77.61 seconds
-------------DONE------------
